In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W22` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W23` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W24` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W25` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W26` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W27` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W28` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W29`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W30`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W31`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W32`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W33`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W34`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.head()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
0,2020-07-16 10:33:59+00:00,2020-07-16 10:34:14+00:00,Preparation/Changeover,None,F4,None,None,KL10,None,Thursday Morning,Temp,0,0.25,2020-30
1,2020-07-11 17:51:00+00:00,2020-07-11 17:53:00+00:00,Factory Closed,None,A4,None,None,KL2,None,Saturday Afternoon,Sanjay,0,1.83,2020-29
2,2020-07-11 21:43:00+00:00,2020-07-11 22:26:00+00:00,Break,None,SE,move to other line,None,KL10,None,Saturday Afternoon,Dummy,0,43.10,2020-29
3,2020-06-06 09:51:00+00:00,2020-06-06 09:54:00+00:00,TBD,None,K4,None,None,KL13,None,Saturday Morning,Tarun,0,2.85,2020-24
4,2020-06-05 18:50:00+00:00,2020-06-06 09:50:00+00:00,Break,None,K4,move to other line,None,KL13,None,Saturday Morning,Tarun,0,900.17,2020-24


In [5]:
data_in_use.shape

(90876, 14)